In [1]:
import pandas as pd
import numpy as np
from lxml import etree


In [2]:
df = pd.read_csv("OpenTripMapDeduplicated.csv")
df.head()

df_geo = pd.read_csv("df_geonames_deduplicated.csv")
df_geo.head()

,Unnamed: 0,name,asciiname,alternatenames,latitude,longitude,feature_class,country_code,id
0,0,Pic de Font Blanca,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.64991,1.53335,T,AD,geonames_id_1
1,1,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,AD,geonames_id_2
2,2,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,AD,geonames_id_3
3,3,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,AD,geonames_id_4
4,4,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,AD,geonames_id_5


In [3]:
#Shuffle for random samples
trip_shuffle = df.sample(frac=1)
geo_shuffle = df_geo.sample(frac=1)

sightlist = df_geo["name"].unique()
print(len(sightlist))
geo_shuffle

1871194


,Unnamed: 0,name,asciiname,alternatenames,latitude,longitude,feature_class,country_code,id
2624891,2631437,Čebine,Cebine,"Cebine,Čebine",46.16867,14.99686,P,SI,geonames_id_2631438
2104492,2110394,Troia Resort Aqualuz Suite Hotel Apartamentos Tr,Troia Resort Aqualuz Suite Hotel Apartamentos Tr,NaN,38.48740,-8.90440,S,PT,geonames_id_2110395
1053714,1056080,Lippó,Lippo,"Lippo,Lippó",45.86278,18.57000,P,HU,geonames_id_1056081
999480,1001840,Saronída,Saronida,"Nkiourntas,Saronida,Saronis,Saronída,Saronís,Γ...",37.74809,23.91059,P,GR,geonames_id_1001841
2652890,2659443,Sarpıncık,Sarpincik,NaN,38.65408,26.40287,P,TR,geonames_id_2659444
...,...,...,...,...,...,...,...,...,...
2583583,2589654,Fröberga,Froberga,"Froberga,Froeberga,Fröberga,Frøberga",59.40000,17.16667,S,SE,geonames_id_2589655
1011017,1013377,Anna Apartments,Anna Apartments,NaN,35.33096,25.24404,S,GR,geonames_id_1013378
2686170,2692725,Soğucak,Sogucak,"Sogucak,Sogulcak,Soğucak,Soğulcak",40.24066,32.62682,P,TR,geonames_id_2692726
2399564,2405492,Barinovo,Barinovo,"Barinovo,Бариново",54.17400,83.29880,P,RU,geonames_id_2405493


In [167]:
#prepare lists and df
longs = geo_shuffle["longitude"].unique()
lats = geo_shuffle["latitude"].unique() 
df_coords = pd.DataFrame(columns=["opentripmap_ID", "geo_ID", "opentripmap_name", "geo_Name", "opentripmap_City", "geo_Country", "match", "geo_long", "trip_long", "geo_lat", "trip_lat", "number_of_matches"])

#reduce df size by only selecting sights where longitude and latitude are in geo names data set
for i, row in trip_shuffle.iterrows():
    if (row["longitude"] in longs) and (row["latitude"] in lats):
        df_coords = df_coords.append({'opentripmap_ID': row["id"],
                                    "opentripmap_name":row["name"],
                                    "opentripmap_City":row["city"],
                                    "trip_long":row["longitude"],
                                    "trip_lat":row["latitude"],
                                     "match":1}, ignore_index=True)

        #Check for sights with the exact same coordinates
for i, row in df_coords.iterrows():
    df_coords["geo_Name"].iloc[i]=df_geo["name"][(df_geo["latitude"]==row["trip_lat"])&(df_geo["longitude"]==row["trip_long"])].array
    df_coords["geo_ID"].iloc[i]=df_geo["id"][(df_geo["latitude"]==row["trip_lat"])&(df_geo["longitude"]==row["trip_long"])].array
    df_coords["number_of_matches"].iloc[i]=len(df_coords["geo_Name"].iloc[i])
    
df_coords

,opentripmap_ID,geo_ID,opentripmap_name,geo_Name,opentripmap_City,geo_Country,match,geo_long,trip_long,geo_lat,trip_lat
0,opentripmap_id_32941,NaN,Auditorium San Barnaba e San Lodovico Pavoni,NaN,Brescia,NaN,1,NaN,10.22618,NaN,45.53569
1,opentripmap_id_107770,NaN,Gasthof Goldener Anker,NaN,Quedlinburg,NaN,1,NaN,11.14960,NaN,51.79150
2,opentripmap_id_44810,NaN,Eekhout Abbey,NaN,Bruges,NaN,1,NaN,3.22952,NaN,51.20520
3,opentripmap_id_56818,NaN,musée Jean-Jacques-Rousseau,NaN,Épinay-sur-Seine,NaN,1,NaN,2.32139,NaN,48.98690
4,opentripmap_id_102591,NaN,Altendorfer Berg,NaN,Einbeck,NaN,1,NaN,9.90503,NaN,51.82280
...,...,...,...,...,...,...,...,...,...,...,...
1657,opentripmap_id_17538,NaN,Synagogen in Dresden,NaN,Dresden,NaN,1,NaN,13.74690,NaN,51.05270
1658,opentripmap_id_23645,NaN,Ostmannturm,NaN,Bielefeld,NaN,1,NaN,8.54018,NaN,52.02890
1659,opentripmap_id_24450,NaN,Villa Sunder-Plaßmann,NaN,Münster,NaN,1,NaN,7.64596,NaN,51.96600
1660,opentripmap_id_121660,NaN,Langscheider Mark,NaN,Balve,NaN,1,NaN,7.92964,NaN,51.33807


In [177]:
#Filter on exact matches
df_finalmente = df_coords[df_coords["matches"]!=0]
df_finalmente = df_finalmente.drop(columns=["geo_Country", "geo_lat", "geo_long"])
print("Number of corner cases:" + str(len(df_finalmente)))

#Manually identify hard matches and hard non matches!!
df_finalmente


Number of corner cases:38


,opentripmap_ID,geo_ID,opentripmap_name,geo_Name,opentripmap_City,match,trip_long,trip_lat,matches
76,opentripmap_id_8590,[geonames_id_1403324],D'Oliphant,[D'Oliphant],Rotterdam,1,4.47000,51.88306,1
106,opentripmap_id_86640,[geonames_id_2625680],Šenčur,[Šenčur],Šenčur,1,14.41972,46.24556,1
114,opentripmap_id_34840,[geonames_id_636324],Playa del Sardinero,[Playa del Sardinero],Santander,1,-3.78483,43.47517,1
133,opentripmap_id_1426,[geonames_id_531926],Raid on Berlin,[Rotes Rathaus],Berlin,1,13.40810,52.51860,1
205,opentripmap_id_94068,[geonames_id_1015522],"Saint Paul Church, Kavala",[Apostolos Pavlos],Kavála,1,24.40811,40.93985,1
209,opentripmap_id_21954,[geonames_id_597042],Playa de las Canteras,[Playa de las Canteras],Las Palmas,1,-15.43170,28.14542,1
265,opentripmap_id_124431,[geonames_id_496212],Kuhstall,[Kuhstall],Sebnitz,1,14.25651,50.92561,1
368,opentripmap_id_28523,[geonames_id_495334],Bürgerpark,[Bürgerpark],Braunschweig,1,10.52284,52.25135,1
401,opentripmap_id_122837,"[geonames_id_62669, geonames_id_74525]",Neustarkenberg,"[Neustarkenberg, Altstarkenberg]",Imst,1,10.75000,47.26667,2
532,opentripmap_id_852,[geonames_id_660064],Cascada Monumental,[Cascada Monumental],Barcelona,1,2.18656,41.38995,1


In [179]:
df_finalmente.to_csv("close_matches_manual_labelling.csv")